In [2]:
from pyspark import SparkConf, SparkContext

In [3]:
conf = SparkConf().setMaster("local[2]").setAppName("MinTemperatures")
sc = SparkContext(conf=conf)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/01 14:41:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
def parseLine(line):
    fields = line.split(",")
    stationID = fields[0]
    entryType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9/5) + 32
    return (stationID, entryType, temperature)

In [6]:
lines = sc.textFile("resources/1800.csv")
lines.take(3)

['ITE00100554,18000101,TMAX,-75,,,E,',
 'ITE00100554,18000101,TMIN,-148,,,E,',
 'GM000010962,18000101,PRCP,0,,,E,']

In [7]:
parsedLines = lines.map(parseLine)
parsedLines.take(5)

[('ITE00100554', 'TMAX', 18.5),
 ('ITE00100554', 'TMIN', 5.359999999999999),
 ('GM000010962', 'PRCP', 32.0),
 ('EZE00100082', 'TMAX', 16.52),
 ('EZE00100082', 'TMIN', 7.699999999999999)]

In [10]:
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
minTemps.take(5)

[('ITE00100554', 'TMIN', 5.359999999999999),
 ('EZE00100082', 'TMIN', 7.699999999999999),
 ('ITE00100554', 'TMIN', 9.5),
 ('EZE00100082', 'TMIN', 8.599999999999998),
 ('ITE00100554', 'TMIN', 23.72)]

In [9]:
stationTemps = minTemps.map(lambda x: (x[0], x[2]))
stationTemps.take(5)

[('ITE00100554', 5.359999999999999),
 ('EZE00100082', 7.699999999999999),
 ('ITE00100554', 9.5),
 ('EZE00100082', 8.599999999999998),
 ('ITE00100554', 23.72)]

In [17]:
minTemps = stationTemps.reduceByKey(lambda x, y: min(x, y))

In [18]:
results = minTemps.collect()

In [19]:
for result in results:
    print(result[0], result[1])

ITE00100554 5.359999999999999
EZE00100082 7.699999999999999


22/02/01 15:50:50 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 957235 ms exceeds timeout 120000 ms
22/02/01 15:50:50 WARN SparkContext: Killing executors is not supported by current scheduler.
